In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import autokeras as ak
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.models import Model
from tensorflow.python.keras.utils.np_utils import to_categorical
from sklearn import metrics
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import load_model

# Train the source domain model

In [6]:

for i in np.arange(0,31,1):
    data = pd.read_csv(r"H:\五类数据37\数据2\NP-snv.csv")
    label = pd.read_csv(r'H:\五类数据37\数据2\标签NP.csv')
    enc = OneHotEncoder().fit_transform(label)
    enc.toarray()
    data.columns = np.double(data.columns)
    label = enc.toarray()
    x = np.array(data)
    y = np.array(label)
    x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.7,random_state = 300)
    y_train = np.array(y_train)
    clf =ak.StructuredDataClassifier(overwrite = True,loss="categorical_crossentropy",metrics="accuracy",max_trials=50)
    clf.fit(x_train,y_train,epochs=300)
    model=clf.export_model()
    tmp = "model_"+str(i)+".h5"
    model.save("tmp")
    loaded_model = load_model(tmp, custom_objects=ak.CUSTOM_OBJECTS)
    loaded_model.summary()
    y_train_predicted = loaded_model.predict(x_train)
    y_test_predicted = loaded_model.predict(x_test)
    for i in range(len(y_train_predicted)):
            max_value=max(y_train_predicted[i])
            for j in range(len(y_train_predicted[i])):
                if max_value==y_train_predicted[i][j]:
                    y_train_predicted[i][j]=1
                else:
                    y_train_predicted[i][j]=0
    for i in range(len(y_test_predicted)):
            max_value=max(y_test_predicted[i])
            for j in range(len(y_test_predicted[i])):
                if max_value==y_test_predicted[i][j]:
                    y_test_predicted[i][j]=1
                else:
                    y_test_predicted[i][j]=0
    from sklearn.metrics import confusion_matrix
    print("\nClassification report for classifier(train):\n\n%s\n"
    % (metrics.classification_report(y_train, y_train_predicted,digits=4)))
    print("Confusion matrix:\n\n%s" % metrics.confusion_matrix(y_train.argmax(axis=1), y_train_predicted.argmax(axis=1)))
    print("\nClassification report for classifier(test):\n\n%s\n"
    % (metrics.classification_report(y_test, y_test_predicted,digits=4)))
    print("Confusion matrix:\n\n%s" % metrics.confusion_matrix(y_test.argmax(axis=1), y_test_predicted.argmax(axis=1)))

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 228)]             0         
                                                                 
 multi_category_encoding (Mu  (None, 228)              0         
 ltiCategoryEncoding)                                            
                                                                 
 normalization (Normalizatio  (None, 228)              457       
 n)                                                              
                                                                 
 dense (Dense)               (None, 256)               58624     
                                                                 
 batch_normalization (BatchN  (None, 256)              1024      
 ormalization)                                                   
                                                             

# Select the source domain model

In [7]:
for i in np.arange(1,11,1):
    data = pd.read_csv(r"H:\五类数据37\数据2\NP-snv.csv")
    label = pd.read_csv(r'H:\五类数据37\数据2\标签NP.csv')
    enc = OneHotEncoder().fit_transform(label)
    enc.toarray()
    data.columns = np.double(data.columns)
    label = enc.toarray()
    x = np.array(data)
    y = np.array(label)
    x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.7,random_state = 300)
    y_train = np.array(y_train)
    tmp = "model_"+str(i)+".h5"
    loaded_model = load_model(tmp, custom_objects=ak.CUSTOM_OBJECTS)
    loaded_model.summary()
    y_train_predicted = loaded_model.predict(x_train)
    y_test_predicted = loaded_model.predict(x_test)
    for i in range(len(y_train_predicted)):
            max_value=max(y_train_predicted[i])
            for j in range(len(y_train_predicted[i])):
                if max_value==y_train_predicted[i][j]:
                    y_train_predicted[i][j]=1
                else:
                    y_train_predicted[i][j]=0
    for i in range(len(y_test_predicted)):
            max_value=max(y_test_predicted[i])
            for j in range(len(y_test_predicted[i])):
                if max_value==y_test_predicted[i][j]:
                    y_test_predicted[i][j]=1
                else:
                    y_test_predicted[i][j]=0
    from sklearn.metrics import confusion_matrix
    print("\nClassification report for classifier(train):\n\n%s\n"
    % (metrics.classification_report(y_train, y_train_predicted,digits=4)))
    print("Confusion matrix:\n\n%s" % metrics.confusion_matrix(y_train.argmax(axis=1), y_train_predicted.argmax(axis=1)))
    print("\nClassification report for classifier(test):\n\n%s\n"
    % (metrics.classification_report(y_test, y_test_predicted,digits=4)))
    print("Confusion matrix:\n\n%s" % metrics.confusion_matrix(y_test.argmax(axis=1), y_test_predicted.argmax(axis=1)))

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 228)]             0         
                                                                 
 multi_category_encoding (Mu  (None, 228)              0         
 ltiCategoryEncoding)                                            
                                                                 
 normalization (Normalizatio  (None, 228)              457       
 n)                                                              
                                                                 
 dense (Dense)               (None, 128)               29312     
                                                                 
 batch_normalization (BatchN  (None, 128)              512       
 ormalization)                                                   
                                                             

# transfer 3:7

In [2]:
# Import target domain data
x_target_sets = pd.read_csv(r".\FM-snv.csv")
y_target_sets = pd.read_csv(r".\label_FM.csv")
enc2 = OneHotEncoder().fit_transform(y_target_sets)
enc2.toarray()
x_target_sets.columns = np.double(x_target_sets.columns)
y_target_sets = enc2.toarray()
x_target_sets = np.array(x_target_sets)
x_t_train, x_t_test,y_t_train,y_t_test = train_test_split(x_target_sets,y_target_sets,test_size = 0.7,random_state = 300) 
y_t_train = np.array(y_t_train)

# Import the source domain model

# Take layers from a previously trained model.

In [9]:
x_target_sets = pd.read_csv(r".\FM-snv.csv")        
y_target_sets = pd.read_csv(r".\label_FM.csv")
enc2 = OneHotEncoder().fit_transform(y_target_sets)
enc2.toarray()
x_target_sets.columns = np.double(x_target_sets.columns)
y_target_sets = enc2.toarray()                                           # Hot coding alone
x_target_sets = np.array(x_target_sets)
x_t_train, x_t_test,y_t_train,y_t_test = train_test_split(x_target_sets,y_target_sets,test_size = 0.7,random_state = 300)    #Divide training set and test set
y_t_train = np.array(y_t_train)
loaded_model = load_model("./model_4.h5", custom_objects=ak.CUSTOM_OBJECTS)       # Import the model
loaded_model.summary()
len(loaded_model.layers)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 228)]             0         
                                                                 
 multi_category_encoding (Mu  (None, 228)              0         
 ltiCategoryEncoding)                                            
                                                                 
 normalization (Normalizatio  (None, 228)              457       
 n)                                                              
                                                                 
 dense (Dense)               (None, 32)                7328      
                                                                 
 batch_normalization (BatchN  (None, 32)               128       
 ormalization)                                                   
                                                             

9

In [31]:
from tensorflow.python.keras.layers.core import Dense, Dropout, Activation, Flatten
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.utils import np_utils
from tensorflow.python.keras import layers
from tensorflow.keras.layers import Dense

In [32]:
model_new = tf.keras.Sequential()        #Take layers from a previously trained model
model_new.add(loaded_model.layers[-9])
model_new.add(loaded_model.layers[-8])
model_new.add(loaded_model.layers[-7])
model_new.add(loaded_model.layers[-6])
model_new.add(loaded_model.layers[-5])
model_new.add(loaded_model.layers[-4])
model_new.add(loaded_model.layers[-3])
# model_new.add(loaded_model.layers[-2])
model_new.add(Dense(3,activation = "softmax"))

In [41]:
model_new.trainable = True                                    # Freezing layers
model_new.layers[-7].trainable = False
model_new.layers[-6].trainable = False
model_new.layers[-5].trainable = True
model_new.layers[-4].trainable = False
model_new.layers[-3].trainable = True
model_new.layers[-2].trainable = True
model_new.layers[-1].trainable = True
model_new.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.00001),     #compile new model
    loss = 'categorical_crossentropy',
    metrics="accuracy"
)
for i in np.arange(1,2,1):
    history = model_new.fit(x_t_train,y_t_train,epochs=500)    #Training new model 
inputs = tf.keras.Input(shape=(228,))
model = tf.keras.Model(inputs)
predicted_t_y = model_new.predict(x_t_test)    #  Use the new model to make predictions
model_new.save(r"H:\五类数据37\tran_model\model_new_autokeras4-2.h5")    # save new model
print(model_new.evaluate(x_t_test, y_t_test))

Epoch 1/500
5/5 [==============================] - 1s 2ms/step - loss: 0.3441 - accuracy: 0.8759
Epoch 2/500
5/5 [==============================] - 0s 2ms/step - loss: 0.3350 - accuracy: 0.8828
Epoch 3/500
5/5 [==============================] - 0s 2ms/step - loss: 0.3431 - accuracy: 0.8897
Epoch 4/500
5/5 [==============================] - 0s 2ms/step - loss: 0.2821 - accuracy: 0.8897
Epoch 5/500
5/5 [==============================] - 0s 2ms/step - loss: 0.3977 - accuracy: 0.8276
Epoch 6/500
5/5 [==============================] - 0s 2ms/step - loss: 0.3743 - accuracy: 0.8828
Epoch 7/500
5/5 [==============================] - 0s 3ms/step - loss: 0.3226 - accuracy: 0.8828
Epoch 8/500
5/5 [==============================] - 0s 2ms/step - loss: 0.3192 - accuracy: 0.8759
Epoch 9/500
5/5 [==============================] - 0s 2ms/step - loss: 0.3127 - accuracy: 0.8897
Epoch 10/500
5/5 [==============================] - 0s 2ms/step - loss: 0.3040 - accuracy: 0.8828
Epoch 11/500
5/5 [===========

# Viewing model metrics

In [46]:
loaded_model = load_model(r"H:\五类数据37\tran_model\model_new_autokeras4-2.h5", custom_objects=ak.CUSTOM_OBJECTS)
loaded_model.summary()
y_train_predicted = loaded_model.predict(x_t_train)
y_test_predicted = loaded_model.predict(x_t_test)
for i in range(len(y_train_predicted)):
        max_value=max(y_train_predicted[i])
        for j in range(len(y_train_predicted[i])):
            if max_value==y_train_predicted[i][j]:
                y_train_predicted[i][j]=1
            else:
                y_train_predicted[i][j]=0
for i in range(len(y_test_predicted)):
        max_value=max(y_test_predicted[i])
        for j in range(len(y_test_predicted[i])):
            if max_value==y_test_predicted[i][j]:
                y_test_predicted[i][j]=1
            else:
                y_test_predicted[i][j]=0
from sklearn.metrics import confusion_matrix
print("\nClassification report for classifier(train):\n\n%s\n"
% (metrics.classification_report(y_t_train, y_train_predicted,digits=4)))
print("Confusion matrix:\n\n%s" % metrics.confusion_matrix(y_t_train.argmax(axis=1), y_train_predicted.argmax(axis=1)))
print("\nClassification report for classifier(test):\n\n%s\n"
% (metrics.classification_report(y_t_test, y_test_predicted,digits=4)))
print("Confusion matrix:\n\n%s" % metrics.confusion_matrix(y_t_test.argmax(axis=1), y_test_predicted.argmax(axis=1)))

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 multi_category_encoding (Mu  (None, 228)              0         
 ltiCategoryEncoding)                                            
                                                                 
 normalization (Normalizatio  (None, 228)              457       
 n)                                                              
                                                                 
 dense (Dense)               (None, 32)                7328      
                                                                 
 batch_normalization (BatchN  (None, 32)               128       
 ormalization)                                                   
                                                                 
 re_lu (ReLU)                (None, 32)                0         
                                                      